## CLEAN DATA

### hiv

In [6]:
import pandas as pd
import numpy as np

path = "data"
indicadores = pd.read_csv(f"{path}/indicadores.csv")

common_cols = {
    "Código": "code", 
    "Nome Município": "name",
}

cols_hiv_mf = {
    "Casos":"hiv_total",
    "Casos M":"hiv_m",
    "Casos F":"hiv_f"#,
    #"Tx Det":"hiv_dr_total",
    #"Tx Det M":"hiv_drm",
    #"Tx Det F":"hiv_drf"
}

cases = pd.DataFrame()
# city code and name
for col_port, col_eng in common_cols.items():
    cases.loc[:,col_eng] = indicadores[col_port]

years_hiv = range(2013,2023)
for year in years_hiv:
    cases[year] = 0
# create panel
cases = (cases.melt(id_vars=["code","name"],
                        value_vars=years_hiv)
          .rename(columns={"variable":"year"})
          .drop("value",axis=1))

sexuality_codes = {
    "Casos Categ. Exp. Hierarq. 10":"hiv_h",
    "Casos Categ. Exp. Hierarq. 20":"hiv_bi",
    "Casos Categ. Exp. Hierarq. 30":"hiv_het"
}

## add columns in 2nd file
def add_rows(final_df, original_df, dict_translations, years, complete=[]):
    for code, group in dict_translations.items():
        #print(code, group)

        cases_1group_all_years = pd.Series()
        for year in years:
            col_original = f"{code} {year}"
            # the column year will have the cases of that year for that group
            # so that I can just melt bellow
            group_year_i = original_df[col_original]
            cases_1group_all_years = pd.concat([cases_1group_all_years, group_year_i],
                                               axis=0)

        for y in complete:
            cases_1group_all_years = pd.concat([cases_1group_all_years,
                                               pd.Series([np.nan]*5657)],
                                               axis=0)
            

        len_cases = len(cases_1group_all_years)
        final_df = (final_df.assign(
                new_col=cases_1group_all_years.values)
                .rename(columns={"new_col":group}))
    return final_df

indicadores_2 = pd.read_csv(f"{path}/indicadores_cont_2.csv")

cases = add_rows(cases, indicadores, cols_hiv_mf, years_hiv)
cases = add_rows(cases, indicadores_2, sexuality_codes, years_hiv)

### hepatites

In [7]:
hepatite_xls = pd.ExcelFile(f"{path}/hepatite.xlsx")

hep_dados = pd.read_excel(hepatite_xls, "DADOS")
hep_dados_cont = pd.read_excel(hepatite_xls, "DADOS CONTINUAÇÃO 1")
#hep_popm = pd.read_excel(hepatite_xls, "Pop.M")
#hep_popf = pd.read_excel(hepatite_xls, "Pop.F")

hep_AB_cols = {
    "Hepatite A":"hep_a_total",
    "Hepatite B":"hep_b_total",
    "Hep B M":"hep_b_m",
    "Hep B F":"hep_b_f",
    "Tx Det Hep A":"hep_a_dr",
    "Tx Det Hep B":"hep_b_dr",
    "Tx Det M Hep A M":"hep_a_dr_m",
    "Tx Det M Hep B M":"hep_b_dr_m",
    "Tx Det F Hep A F":"hep_a_dr_f",
    "Tx Det F Hep B F":"hep_b_dr_f"
}
hep_C_cols = {
    "Hepatite C":"hep_c_total",
    "Tx Det Hep C":"hep_c_dr",
    "Tx Det M Hep C M":"hep_c_dr_m",
    "Tx Det F Hep C F":"hep_c_dr_f"
}

hep_populacao = pd.read_excel(hepatite_xls, "População")
population = hep_populacao[["Código","Nome Município",2016]]
population = population.rename(columns={"Código":"code",
                                "Nome Município":"name",
                                2016:"population"})

years_hep = range(2013, 2021)#13-20

cases = add_rows(cases, hep_dados, hep_AB_cols, years_hep, complete=[2021,2022])
cases = add_rows(cases, hep_dados_cont, hep_C_cols, years_hep, complete=[2021,2022])

#### add prep data, controls and region info

In [8]:
cases = cases.set_index("code")
cases.loc[:, "region_type"] = "city"
cases.loc[cases.index < 100, "region_type"] = "state"
cases.loc[cases.index <= 10, "region_type"] = "region"
cases.loc[55, "region_type"] = "brazil"
cases = cases.reset_index()

# add percent treat

import pandas as pd
prep = pd.DataFrame()
for year in range(2018, 2023):
    df = pd.read_csv(f"data/prep{year}.csv")
    df["year"] = year
    prep = pd.concat([prep, df])
prep_data = prep.groupby(["city_code", "year"]).tail(1)

new_cases = (prep_data.set_index(["city_code","year"])
  .sort_values(["city_code","year"])
    .groupby("city_code")
    .agg( {"following":"diff"} )
    .reset_index()
    .rename(columns={"following":"new_users"}))
prep_data = prep_data.merge(new_cases, left_on=["city_code","year"],
                            right_on=["city_code","year"],how="left")

prep_data.loc[prep_data["year"] == 2018, "new_users"] = prep_data["following"]

cases = cases.merge(prep_data, left_on=["code","year"],
                right_on=["city_code","year"],
                how="left")
cases.loc[cases["hiv_h"] == 0, "pct_new_users"] = 0
cases.loc[cases["hiv_h"] != 0, "pct_new_users"] = cases["new_users"]/cases["hiv_h"]
cases.loc[cases["new_users"] < 0, "pct_new_users"] = 0

cases.loc[cases["hiv_h"] == 0, "pct_following"] = 0
cases.loc[cases["hiv_h"] != 0, "pct_following"] = cases["following"]/cases["hiv_h"]

cases = cases.query("code != 0")

cases = cases.merge(population,
            left_on=["code","name"],
            right_on=["code","name"],
            how="left")

## CONTROLS
cities = cases.query("region_type == 'city'")
cities["state_code"] = cities["code"].astype(str).str.slice(0,2)

## ieps = institute for health policy studies
ieps = pd.read_csv("data/ieps.csv")
rename_ieps = {
    "codmun":"code",
    "ano":"year",
    "desp_tot_saude_pc_mun":"health_expend",
    "gasto_pbf_pc_def":"cash_transfer_expend",
    "pib_cte_pc":"gdpp",
    "pct_pop20a24":"pop20_24"
}
ieps = ieps.rename(columns=rename_ieps)
ieps = ieps.replace(' ',pd.NA)
ieps = ieps[["code","year","health_expend",
             "cash_transfer_expend","gdpp","pop20_24"]]

numeric_cols = ["health_expend","cash_transfer_expend",
                "gdpp","pop20_24"]
for c in numeric_cols:
    ieps.loc[ieps[c].notna(),c] = ieps.loc[ieps[c].notna(),c].str.replace(",",".").astype(float).round(2)

cities = cities.merge(ieps, on=["code","year"], how="left")

## COVID DATA
covid = pd.read_csv("data/covid.csv")
covid = covid.query("place_type == 'city'")
covid["date"] = pd.to_datetime(covid["date"])
covid = covid.rename(columns={"date":"year"})
covid = covid.sort_values(["state","city","year"])
covid = covid.set_index(["state","city"])
covid_cases = (covid.groupby(["state", "city",covid["year"].dt.year])
                 .agg({"confirmed":"last"}).reset_index())
covid_cases = covid_cases.rename(columns={"confirmed":"covid_cases"})
cities = cities.merge(covid_cases, left_on=["state_code","name","year"],
                          right_on=["state","city","year"], how="left")
cities.loc[cities["covid_cases"].isnull(), "covid_cases"] = 0

## MERGE WITH REGION INFO

ibge = pd.read_excel("data/regioes_ibge.xls")
ibge["state"] = ibge["CD_GEOCODI"].astype("str").str.slice(0, 2)

cities.loc[:,"state"] = cities["code"].astype("str").str.slice(0, 2)

ibge = ibge[["nome_mun", "state", "nome_rgi", "cod_rgi"]]
ibge = ibge.rename(columns={"nome_mun":"name",
                            "nome_rgi":"region_name",
                            "cod_rgi":"region_code"})
cities = cities.merge(ibge, on=["state","name"],
                        how="inner")
cities.to_csv("data/cities.csv", mode="w", index=False)

/tmp/ipykernel_19728/2675067475.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities["state_code"] = cities["code"].astype(str).str.slice(0,2)


In [21]:
#regions
import numpy as np
regions = (cities.set_index(["region_code","region_name","year"])
        .groupby(["region_code","region_name","year"])
        .agg({"hiv_total":"sum",
              "hiv_h":"sum",
              "hiv_het":"sum",
              "hiv_bi":"sum",
              "hiv_f":"sum",
              "hiv_m":"sum",
              "hep_b_total":"sum",
              "hep_b_m":"sum",
              "hep_b_f":"sum",
              "following":"sum",
              "loss_of_follow":"sum",
              "new_users":"sum",
              "population":"sum"
             })
        .reset_index())

regions.loc[regions["hiv_h"] == 0, "pct_new_users"] = 0
regions.loc[regions["hiv_h"] != 0, "pct_new_users"] = regions["new_users"]/regions["hiv_h"]
regions.loc[regions["new_users"] < 0, "pct_new_users"] = 0

regions.loc[regions["hiv_h"] == 0, "pct_following"] = 0
regions.loc[regions["hiv_h"] != 0, "pct_following"] = regions["following"]/regions["hiv_h"]

regions = regions.set_index("region_code")

def average_by_region(var):
    return (cities.query(f"{var}.notna()").groupby(["region_code","year"])
            .apply(lambda x: np.average(x[var], weights=x.population))
    .reset_index().rename(columns={0:var} ))

control_vars = ["health_expend","cash_transfer_expend","gdpp","pop20_24","covid_cases"]

for var in control_vars:
    ## compute the average of the outcome by region
    ## ieps has no data for 2022
    var_df = average_by_region(var)
    regions = regions.merge(var_df, on=["region_code","year"],how="left")
    
regions["state_code"] = regions["region_code"].astype(str).str.slice(0,2)
regions.to_csv("data/regions.csv", mode="w", index=False)